## Connect to a DAOS pool


### Prerequisites

- Make sure the pool "python-pool" exists.

    ```
    $ dmg system list-pools
    Pool        Size   State Used Imbalance Disabled
    ----        ----   ----- ---- --------- --------
    python-pool 3.9 TB Ready 0%   0%        0/96
    ```
- Create the container "py-testcont" in the above pool.
    ```
    $ daos cont create $pool_name $cont_name -t PYTHON
    Successfully created container 01571915-f3bc-472e-ac42-c63f02ed54fd
    Container UUID : 01571915-f3bc-472e-ac42-c63f02ed54fd
    Container Label: py-testcont
    Container Type : PYTHON
    ```


In [ ]:
# NOTE: UPDATE YOUR PATH HERE!!!
# Manually import the DAOS python module
# Find the pydaos module by: sudo find / -name "*pydaos*"
# Usually it is at /usr/lib64/python3.6/site-packages/pydaos

import sys
sys.path.append('/usr/lib64/python3.6/site-packages')  # depend on your path
import pydaos

For the remaining code, please refer to: https://docs.daos.io/v2.4/user/python/#daos-dictionaries. Only the `pydaos.pydaos_core` is used. 

In [ ]:
POOL_NAME="python-pool"
CONT_LABEL="py-testcont"

# A DAOS container 
dcont = pydaos.DCont(pool=POOL_NAME, cont=CONT_LABEL)
print(dcont)

Start a Redis stream client.

In [ ]:
import redis

REDIS_HOST = 'localhost'
REDIS_PORT = 6379
STREAM_NAME = "ejfat"

redis_conn = redis.Redis(host=REDIS_HOST, port=REDIS_PORT)

Create a DAOS python dictionary object.

In [ ]:
DDICT_NAME = "ping-dict"

# dd = dcont.dict(DDICT_NAME, {})  # create. Can execute only once.
dd = dcont.get(DDICT_NAME)
len1 = len(dd)
print(len1)


Read from the Redis stream and write to the DAOS dictionary object. Remember to run `python demo_writer.py` at the same time.

In [ ]:
start_id = '0'  # "$" means only to read the new messages

# Block time in milliseconds (0 means non-blocking)
block_time = 20

cnt = 0
while cnt != 100:  # run for ~block_time * cnt ms 
    print(f"cnt: {cnt}")
    # Use xread with the block option
    response = redis_conn.xread({STREAM_NAME: start_id}, block=block_time)

    # Check if there are new messages
    if not response:
        # No new messages within the block time
        # print("No new messages within the block time.")
        cnt += 1
        continue

    for stream, messages in response:
        for message in messages:
            message_id, message_values = message
            dd[message_id] = message_values[b'data']
            print(f"  message_id {message_id} of entry_id {message_values[b'entry_id']}")  # print idx
            # print(message_values[b'data'])

            # Acknowledge the processed message
            redis_conn.xack(STREAM_NAME, '', message_id)  # Note the empty consumer name

            # Delete the processed message from the stream
            redis_conn.xdel(STREAM_NAME, message_id)

    cnt += 1

# Close the Redis connection (optional)
redis_conn.close()

Check the DAOS dictionary object length again. The length should be different.

In [ ]:
print(len(dd) - len1)
# print(dd.dump())

for key in dd:
    print(key)

### Refs
- https://docs.daos.io/v2.4/user/python/
- https://github.com/daos-stack/daos/tree/release/2.4/src/client/pydaos/raw
- https://github.com/daos-stack/daos/blob/release/2.4/src/client/pydaos/raw/daos_api.py
